# Installation

In [1]:
!pip install --upgrade pip
!pip install cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.5/576.5 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import Packages

In [2]:
import cirq
import numpy as np

print('cirq version',cirq.__version__)
print('numpy version',np.__version__)


cirq version 1.0.0
numpy version 1.21.6


# Quantum Fourier Transform

In [3]:
## This code is cloned from the work done in https://github.com/Apress/quantum-machine-learning-python/blob/main/Chapter_4/listing4_1/quantum_fourier_transform.py 
class QFT:
    """
    Quantum Fourier Transform
    Builds the QFT circuit iteratively 
    """

    def __init__(self, signal_length=16,
                 basis_to_transform='',
                 validate_inverse_fourier=False,
                 qubits=None):
        
        self.signal_length = signal_length
        self.basis_to_transform = basis_to_transform
        
        if qubits is None:
            self.num_qubits = int(np.log2(signal_length))
            self.qubits = [cirq.LineQubit(i) for i in range(self.num_qubits)]
        else:
            self.qubits = qubits
            self.num_qubits = len(self.qubits)

        self.qubit_index = 0
        self.input_circuit = cirq.Circuit()

        self.validate_inverse_fourier = validate_inverse_fourier
        self.circuit = cirq.Circuit()
        # if self.validate_inverse_fourier:
        self.inv_circuit = cirq.Circuit()

        for k, q_s in enumerate(self.basis_to_transform):
            if int(q_s) == 1:
                # Change the qubit state from 0 to 1 
                self.input_circuit.append(cirq.X(self.qubits[k]))

    def qft_circuit_iter(self):

        if self.qubit_index > 0:
            # Apply the rotations on the prior qubits
            # conditioned on the current qubit
            for j in range(self.qubit_index):
                diff = self.qubit_index - j + 1
                rotation_to_apply = 2.0 / (2.0 ** diff)
                self.circuit.append(cirq.CZ(self.qubits[self.qubit_index],
                                            self.qubits[j]) ** rotation_to_apply)
        # Apply the Hadamard Transform 
        # on current qubit
        self.circuit.append(cirq.H(self.qubits[self.qubit_index]))
        # set up the processing for next qubit
        self.qubit_index += 1

    def qft_circuit(self):

        while self.qubit_index < self.num_qubits:
            self.qft_circuit_iter()
            # See the progression of the Circuit built
            print(f"Circuit after processing Qubit: {self.qubit_index - 1} ")
            print(self.circuit)
        # Swap the qubits to match qft definititon
        self.swap_qubits()
        print("Circuit after qubit state swap:")
        print(self.circuit)
        # Create the inverse Fourier Transform Circuit
        self.inv_circuit = cirq.inverse(self.circuit.copy())

    def swap_qubits(self):
        for i in range(self.num_qubits // 2):
            self.circuit.append(cirq.SWAP(self.qubits[i], self.qubits[self.num_qubits - i - 1]))

    def simulate_circuit(self):
        sim = cirq.Simulator()
        result = sim.simulate(self.circuit)
        return result

# Simulation

In [4]:
signal_length=16
basis_to_transform='0000'
validate_inverse_fourier=False

# Instantiate QFT Class
_qft_ = QFT(signal_length=signal_length,
            basis_to_transform=basis_to_transform,
            validate_inverse_fourier=validate_inverse_fourier)


# Build the QFT Circuit
_qft_.qft_circuit()

# Create the input Qubit State

if len(_qft_.input_circuit) > 0:
    _qft_.circuit = _qft_.input_circuit + _qft_.circuit

if _qft_.validate_inverse_fourier:
    _qft_.circuit += _qft_.inv_circuit

print("Combined Circuit")
print(_qft_.circuit)
# Simulate the circuit

output_state = _qft_.simulate_circuit()
# Print the Results
print(output_state)

Circuit after processing Qubit: 0 
0: ───H───
Circuit after processing Qubit: 1 
0: ───H───@───────────
          │
1: ───────@^0.5───H───
Circuit after processing Qubit: 2 
                  ┌───────┐
0: ───H───@─────────@─────────────────────
          │         │
1: ───────@^0.5────H┼─────────@───────────
                    │         │
2: ─────────────────@^0.25────@^0.5───H───
                  └───────┘
Circuit after processing Qubit: 3 
                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@─────────@───────────────@──────────────────────────────────
          │         │               │
1: ───────@^0.5────H┼──────────@────┼────────────@─────────────────────
                    │          │    │            │
2: ─────────────────@^0.25─────@^0.5┼───────────H┼─────────@───────────
                                    │            │         │
3: ─────────────────────────────────@^(1/8)──────@^0.25────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘
